In [4]:
import pandas as pd
import numpy as np
import os

# shipments

1) user_id - id клиента в системе

2) ship_address_id - foreign key к addresses

3) shipment_id - foreign key к line_items

4) order_id - id заказа (в одной доставке может быть несколько заказов)

5) order_created_at - дата начала оформления заказа

6) order_completed_at - дата оформления заказа

7) shipment_starts_at - дата начала слота доставки

8) retailer - название ритейлера

9) order_state - статус заказа

10) shipment_state - статус доставки

11) city_name - название города

12) store_id - id конкретного магазина

13) total_cost - фактические затраты на доставку с учетом промо на доставку

14) rate - оценка заказа

15) dw_kind - тип доставки

16) promo_total - промо на товары

17) total_weight - общий вес заказа

18) platform - платформа

19) os - операционная система

20) shipped_at - дата доставки заказа

In [62]:
data = []
for file in sorted(os.listdir('shipments/')):
    data.append(file)
shipments = pd.read_csv('shipments/' + data[0])
for d in data[1:]:
    shipments = pd.concat([shipments, pd.read_csv(str('shipments/' + d))]).reset_index(drop=True)

user = pd.read_csv('misc/user_profiles_Nastya.csv')

data = []
for file in sorted(os.listdir('line_items01/')):
    data.append(file)
line01 = pd.read_csv('line_items01/' + data[0])
for d in data[1:]:
    line01 = pd.concat([line01, pd.read_csv(str('line_items01/' + d))])

In [63]:
ship_user = pd.merge(shipments, user, on='user_id', how='left')

ship_user['age'] = ship_user['age'].replace(0, np.nan)

In [64]:
ship_user_line01 = pd.merge(ship_user, line01, on='shipment_id', how='left')

In [65]:
brands_stats = ship_user_line01['brand_name'].value_counts() 

big_brands = brands_stats[brands_stats > 3000].index

ship_user_line01.at[~ship_user_line01['brand_name'].isin(big_brands), 'brand_name'] = np.nan

In [10]:
ship_user_line01.shape

(4409955, 34)

In [11]:
dates = ['order_created_at', 'order_completed_at', 'shipped_at', 'shipment_starts_at', 'bdate', 'created_at']

cats = []
for col in ship_user_line01.columns:
    if col not in dates:
        if ship_user_line01[col].dtype == 'object':
            cats.append(col)
            print(col, len(ship_user_line01[col].value_counts()))

retailer 46
s.order_state 4
shipment_state 7
s.city_name 97
dw_kind 3
platform 2
os 6
gender 2
brand_name 130
pricer 4


In [13]:
mess = pd.read_csv('messages/messages.csv')
actions = pd.read_csv('messages/actions-labeled.csv').drop('Unnamed: 0', axis=1)
mess_act = pd.merge(mess, actions, left_on='action_id', right_on='id')

mess_act.head()

,sent,action_id,user_id,id,subject,body,type,type_subject
0,1.588847e+09,1512.0,1775692,1512,Сегодня готовим брауни с M&M's 😋 Приступим?,Видео рецепт и все необходимые ингредиенты уже...,email,коммуникация
1,1.588847e+09,1512.0,1775687,1512,Сегодня готовим брауни с M&M's 😋 Приступим?,Видео рецепт и все необходимые ингредиенты уже...,email,коммуникация
2,1.588847e+09,1512.0,1775684,1512,Сегодня готовим брауни с M&M's 😋 Приступим?,Видео рецепт и все необходимые ингредиенты уже...,email,коммуникация
3,1.588847e+09,1512.0,1775678,1512,Сегодня готовим брауни с M&M's 😋 Приступим?,Видео рецепт и все необходимые ингредиенты уже...,email,коммуникация
4,1.588847e+09,1512.0,1775660,1512,Сегодня готовим брауни с M&M's 😋 Приступим?,Видео рецепт и все необходимые ингредиенты уже...,email,коммуникация


In [14]:
# ship_user_line01_mess = pd.merge(ship_user_line01, mess_act, on=['user_id'], how='left')

KeyboardInterrupt: 

In [27]:
ship_user_line01.columns

Index(['user_id', 'ship_address_id', 'shipment_id', 'order_id',
       'order_created_at', 'order_completed_at', 'shipment_starts_at',
       'retailer', 's.order_state', 'shipment_state', 's.city_name',
       's.store_id', 'total_cost', 'rate', 'dw_kind', 'promo_total',
       'total_weight', 'platform', 'os', 'shipped_at', 'gender', 'bdate',
       'age', 'master_category_id', 'created_at', 'price', 'discount',
       'item_id', 'quantity', 'product_id', 'brand_name', 'pricer', 'replaced',
       'cancelled', 'month'],
      dtype='object')

In [28]:
ship_user_line01['shipment_state'].value_counts()

shipped          4048312
canceled          351854
ready                503
collecting           486
ready_to_ship        412
shipping             304
pending               28
Name: shipment_state, dtype: int64

In [33]:
ship_user_line01['shipment_time'] = (pd.to_datetime(ship_user_line01['shipped_at']) - pd.to_datetime(ship_user_line01['shipment_starts_at']))

0    01:08:54
1    01:10:40
2    01:10:40
3    01:10:40
4    01:10:40
5    01:10:40
6    01:10:40
7    01:10:40
8    01:10:40
9    01:10:40
10   01:10:40
11   01:10:40
12   01:10:40
13   01:10:40
14   01:10:40
15   01:10:40
16   01:10:40
17   01:10:40
18   01:10:40
19   01:10:40
20   01:10:40
21   01:10:40
22   01:10:40
23   01:10:40
24   01:10:40
25   01:10:40
26   01:10:40
27   01:10:40
28   01:10:40
29   01:10:40
dtype: timedelta64[ns]

In [59]:
users_cancelled = ship_user_line01[ship_user_line01['shipment_state'] == 'canceled'].groupby('user_id')['shipment_state'].agg('count')

users_shipped = ship_user_line01[ship_user_line01['shipment_state'] == 'shipped'].groupby('user_id')['shipment_state'].agg('count')

ship_user_line01 = ship_user_line01.join(users_shipped, rsuffix='_shipped_count')
ship_user_line01 = ship_user_line01.join(users_cancelled, rsuffix='_cancelled_count')

In [61]:
df['shipment_state_shipped_count'].value_counts()

shipped          4048312
canceled          351854
ready                503
collecting           486
ready_to_ship        412
shipping             304
pending               28
Name: shipment_state_shipped_count, dtype: int64

In [20]:
ship_user_line01['order_completed_at'] = pd.to_datetime(ship_user_line01['order_completed_at'])

In [21]:
ship_user_line01['month'] = ship_user_line01['order_completed_at'].dt.to_period('M')

In [23]:
ship_user_line01['month'].value_counts()

2020-01    1589155
2020-02     701690
2020-05     441274
2020-08     428807
2020-07     419985
2020-06     414987
2020-04     258195
2020-03     155755
2019-12        107
Freq: M, Name: month, dtype: int64

In [26]:
ship_user_line01['month']

0          2020-02
1          2020-01
2          2020-01
3          2020-01
4          2020-01
            ...   
4409950    2020-08
4409951    2020-08
4409952    2020-08
4409953    2020-08
4409954    2020-08
Name: month, Length: 4409955, dtype: period[M]

In [22]:
ship_user_line01.groupby(['user_id', 'month']).sum().reset_index().groupby('user_id').rolling(3, on='month',  min_periods=1).total_weight.sum()

KeyboardInterrupt: 

# model

In [69]:
old_ids = set(sample_submission['Id']) & set(train['phone_id'])

sample_submission.at[sample_submission['Id'].isin(old_ids),'Predicted'] = 1

sample_submission.Predicted = sample_submission.Predicted.astype(int)

In [146]:
train = pd.read_csv('train/train.csv')
sample_submission = pd.read_csv('sample_submission.csv', sep=';')

In [147]:
sample_submission.Predicted = 1

In [149]:
import random
random.seed = 0
randsample = random.sample(range(len(sample_submission)), 200)

In [150]:
sample_submission.loc[randsample, 'Predicted'] = 0

In [151]:
sample_submission.shape

(214609, 2)

In [152]:
sample_submission.sum()

Id           55694902038
Predicted         214409
dtype: int64

In [153]:
sample_submission.set_index('Id').to_csv('submission3.csv')

In [156]:
train['target'].mean()

0.5004027334129026

In [42]:
addr = pd.read_csv('misc/addresses.csv')

In [64]:
train_full = pd.merge(train, addr, left_on='phone_id', right_on='phone_id', how='left')

In [71]:
submission_full = pd.merge(sample_submission, addr, left_on='Id', right_on='phone_id')

In [77]:
train_ship = pd.merge(train_full, shipments, left_on='id', right_on='ship_address_id')
sub_ship = pd.merge(submission_full, shipments, left_on='id', right_on='ship_address_id')

In [93]:
train_ship.columns

Index(['phone_id', 'order_completed_at_x', 'target', 'id', 'user_id',
       'ship_address_id', 'shipment_id', 'order_id', 'order_created_at',
       'order_completed_at_y', 'shipment_starts_at', 'retailer',
       's.order_state', 'shipment_state', 's.city_name', 's.store_id',
       'total_cost', 'rate', 'dw_kind', 'promo_total', 'total_weight',
       'platform', 'os', 'shipped_at'],
      dtype='object')

In [ ]:
test_ship = train_test_split(...)

In [ ]:
train_columns = [ 'retailer', 'shipment_state', 's.city_name', 
       'total_cost', 'rate', 'dw_kind', 'promo_total', 'total_weight',
       'platform', 'os']

In [ ]:
X = train_ship.drop('target',axis=1)
test = test_ship.drop('target',axis=1)

y = train_ship.target

In [29]:
warnings.simplefilter('ignore')
import time
from numba import jit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn import metrics
from sklearn.model_selection import KFold
import gc
from sklearn.utils import shuffle

@jit
def fast_auc(y_true, y_prob):
    """
    fast roc_auc computation: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76013
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc
def eval_auc(y_true, y_pred):
    """
    Fast auc eval function for lgb.
    """
    return 'auc', fast_auc(y_true, y_pred), True
def group_mean_log_mae(y_true, y_pred, types, floor=1e-9):
    """
    Fast metric computation for this competition: https://www.kaggle.com/c/champs-scalar-coupling
    Code is from this kernel: https://www.kaggle.com/uberkinder/efficient-metric
    """
    maes = (y_true - y_pred).abs().groupby(types).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()
def plot_importance(result_dict, file=None):
    feature_importance = result_dict['feature_importance']
    cols = result_dict['top_columns']
    best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

    plt.figure(figsize=(16, 12))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LGB Features (avg over folds)')
    if file is not None:
        plt.savefig(file)
def train_model_classification_vb(X, X_test, y, params, folds, model_type='lgb', eval_metric='auc', columns=None,
                                  plot_feature_importance=True, model=None,
                                  verbose=100, early_stopping_rounds=200, n_estimators=500, splits=None,
                                  averaging='usual', n_jobs=-1, groups=None,
                                  train_1_sample_coef=None, train_0_sample_coef=None,
                                  categorial_columns=None, categorial_encoder=None
                                  ):
    """
    A function to train a variety of classification models.
    Returns dictionary with oof predictions, test predictions, scores and, if necessary, feature importances.
    :params: X - training data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: X_test - test data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: y - target
    :params: folds - folds to split data
    :params: model_type - type of model to use
    :params: eval_metric - metric to use
    :params: columns - columns to use. If None - use all columns
    :params: plot_feature_importance - whether to   plot feature importance of LGB
    :params: model - sklearn model, works only for "sklearn" model type
    """
    columns = X.columns if columns is None else columns
    n_splits = folds.n_splits if splits is None else splits
    if X_test is not None:
        X_test = X_test[columns]

    # to set up scoring parameters
    metrics_dict = {'auc': {'lgb_metric_name': eval_auc,
                            'catboost_metric_name': 'AUC',
                            'sklearn_scoring_function': metrics.roc_auc_score},
                    }

    result_dict = {}
    if averaging == 'usual':
        # out-of-fold predictions on train data
        oof = np.zeros((len(X), 1))

        # averaged predictions on train data
        if X_test is not None:
            prediction = np.zeros((len(X_test), 1))

    elif averaging == 'rank':
        # out-of-fold predictions on train data
        oof = np.zeros((len(X), 1))

        # averaged predictions on train data
        if X_test is not None:
            prediction = np.zeros((len(X_test), 1))

    # list of scores on folds
    scores = []
    feature_importance = pd.DataFrame()

    # split and train on folds
    if groups is None:
        splitter = folds.split(X, y)
    else:
        splitter = folds.split(X, y, groups=groups)
    for fold_n, (train_index, valid_index) in enumerate(splitter):
        gc.collect()
        if fold_n < folds.n_splits - n_splits:
            continue
        print(f'Fold {fold_n + 1} started at {time.ctime()}')
        if type(X) == np.ndarray:
            X_train, X_valid = X[columns][train_index], X[columns][valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
        else:
            X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        # encode categorial features
        if categorial_encoder is not None:
            print('Encoding categorical features')
            assert categorial_columns is not None
            encoder = categorial_encoder(verbose=1, cols=categorial_columns)
            encoder.fit(X_train, y_train)
            X_train = encoder.transform(X_train)
            X_valid = encoder.transform(X_valid)
            X_test = encoder.transform(X_test)

        # down/upsample train dataset
        if train_0_sample_coef is not None or train_1_sample_coef is not None:
            assert NotImplementedError
            if type(X_train) is np.ndarray:
                assert NotImplementedError
                # train = np.hstack([X_train.values, y_train.reshape((-1,1))])
            else:
                X_train['target'] = y_train

            train_0 = X_train[X_train.target == 0]
            train_1 = X_train[X_train.target == 1]
            if train_0_sample_coef is not None:
                train_0 = train_0.sample(int(train_0.shape[0] * train_0_sample_coef),
                                         random_state=params['random_state'], replace=True)
            if train_1_sample_coef is not None:
                train_1 = train_1.sample(int(train_1.shape[0] * train_1_sample_coef),
                                         random_state=params['random_state'], replace=True)
            train = pd.concat([train_0, train_1], axis=0)
            train = shuffle(train, random_state=params['random_state'])
            X_train = train.drop(['target'], axis=1)
            y_train = train.target
            del train_0
            del train_1
            del train

        if model_type == 'lgb':
            model = lgb.LGBMClassifier(**params, n_estimators=n_estimators, n_jobs=n_jobs)
            model.fit(X_train, y_train,
                      eval_set=[(X_train, y_train), (X_valid, y_valid)],
                      eval_metric=metrics_dict[eval_metric]['lgb_metric_name'],
                      verbose=verbose, early_stopping_rounds=early_stopping_rounds)
            gc.collect()
            y_pred_valid = model.predict_proba(X_valid, num_iteration=model.best_iteration_)[:, 1]
            if X_test is not None:
                y_pred = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]

        if model_type == 'xgb':
            if columns is None:
                feature_names = X.columns
            else:
                feature_names = columns
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=feature_names)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=feature_names)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=n_estimators, evals=watchlist,
                              early_stopping_rounds=early_stopping_rounds, verbose_eval=verbose, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=feature_names),
                                         ntree_limit=model.best_ntree_limit)
            if X_test is not None:
                y_pred = model.predict(xgb.DMatrix(X_test, feature_names=feature_names),
                                       ntree_limit=model.best_ntree_limit)

        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)

            y_pred_valid = model.predict_proba(X_valid)[:, 1].reshape(-1, )
            score = metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid)
            print(f'Fold {fold_n}. {eval_metric}: {score:.4f}.')
            print('')
            if X_test is not None:
                y_pred = model.predict_proba(X_test)[:, 1]

        if model_type == 'keras':
            from keras.models import Model
            from keras.callbacks import Callback, EarlyStopping

            kmodel: Model = model()

            my_callbacks = [EarlyStopping(monitor='aucroc', patience=early_stopping_rounds, verbose=1, mode='max')]

            keras_params = {k: v for k, v in params.items() if k not in ('random_state',)}
            kmodel.fit(X_train, y_train, validation_data=(X_valid, y_valid), **keras_params, callbacks=my_callbacks)
            predict_params = {k: v for k, v in params.items() if k in ['batch_size', 'verbose', 'steps', 'callbacks',
                                                                       'max_queue_size', 'workers',
                                                                       'use_multiprocessing']}
            y_pred_valid = kmodel.predict(X_valid) #, **predict_params
            score = metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid)
            print(f'Fold {fold_n}. {eval_metric}: {score:.4f}.')
            if X_test is not None:
                y_pred = kmodel.predict(X_test)[:,0]

        if model_type == 'cat':
            model = CatBoostClassifier(iterations=n_estimators, **params)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=categorial_columns,
                      use_best_model=True,
                      verbose=True)

            y_pred_valid = model.predict_proba(X_valid)[:, 1]
            if X_test is not None:
                y_pred = model.predict_proba(X_test)[:, 1]

        if averaging == 'usual':

            oof[valid_index] = y_pred_valid.reshape(-1, 1)
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
            if X_test is not None:
                prediction += y_pred.reshape(-1, 1)

        elif averaging == 'rank':

            oof[valid_index] = y_pred_valid.reshape(-1, 1)
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
            if X_test is not None:
                prediction += pd.Series(y_pred).rank().values.reshape(-1, 1)

        if model_type == 'lgb' and plot_feature_importance:
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    if X_test is not None:
        prediction /= n_splits

    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))

    result_dict['oof'] = oof
    if X_test is not None:
        result_dict['prediction'] = prediction
    result_dict['scores'] = scores

    if model_type == 'lgb':
        if plot_feature_importance:
            feature_importance["importance"] /= n_splits
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            result_dict['feature_importance'] = feature_importance
            result_dict['top_columns'] = cols
            plot_importance(result_dict)

    return result_dict

In [ ]:
from sklearn.model_selection import GroupKFold

NFOLDS = 6
folds = GroupKFold(n_splits=NFOLDS)

In [ ]:
params = {
          'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.01,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }
train_options = {
        "model_type":'lgb',
        "params": params,
        "eval_metric":'auc',
        'early_stopping_rounds': 500,
        'n_estimators': 1000,
        'averaging': 'usual',
        'use_groups': False,
        'fold_name': folds.__class__.__name__,
        'n_splits': NFOLDS
    }

In [ ]:
result_dict = train_model_classification_vb( 
                                             X=X, X_test=test, columns=train_columns,
                                             y=y, params=params, folds=folds,
                                             model_type=train_options['model_type'], 
                                             plot_feature_importance=True,
                                             verbose=500, early_stopping_rounds=train_options['early_stopping_rounds'],
                                             n_estimators=train_options['n_estimators'], 
                                             averaging=train_options['averaging'],
                                             n_jobs=-1)

In [ ]:
pd.DataFrame(result_dict['oof'], columns=['target'], index=X.index).to_csv(f'{data_path}/oof_lightgbm.csv')

pd.DataFrame(result_dict['prediction'], columns=['isFraud'], index=test.index).to_csv(f'{data_path}/prediction_lightgbm-work-1.csv')

sample_submission = pd.read_csv(data_path + '/sample_submission.csv').set_index('TransactionID')

sub1 = pd.DataFrame(result_dict['prediction'], columns=['isFraud'], index=test.index)

sample_submission['target'] = sub1#*0.5 + sub2*0.25 + sub3*0.25

sample_submission.to_csv(f'{data_path}/lightgbm-work-1_sub.csv')